In [ ]:
pip install optuna

In [ ]:
pip install xgboost

In [3]:
## Importing libraries
import optuna
import pandas as pd
import numpy as np
from Amex_Metric import amex_metric
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

## Reading the data
train = pd.read_csv('/home/ec2-user/SageMaker/Analytics_Data_Science/American_Express/Evan/amex_train_payment_spend_final.csv')
test = pd.read_csv('/home/ec2-user/SageMaker/Analytics_Data_Science/American_Express/Evan/amex_test_payment_spend_final.csv')

Matplotlib is building the font cache; this may take a moment.


In [4]:
train.head()

,customer_ID,P_2_mean,P_2_median,P_2_correlation,P_3_mean,P_2_sum,S_25_mean,S_25_sum,S_25_std,S_25_mad,S_25_data_range,S_25_iqr,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.9336,0.9385,-0.438767,0.6800,12.140,0.9746,12.670,0.002504,0.001802,0.009280,0.001465,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.9000,0.9050,-0.854416,0.5670,11.695,0.9756,12.680,0.002622,0.001802,0.008790,0.001465,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.8784,0.8850,-0.109422,0.6180,11.420,0.9740,12.664,0.002858,0.002329,0.009766,0.003418,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.5990,0.5980,0.953176,0.6110,7.785,0.9746,12.670,0.002941,0.002403,0.008790,0.004395,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.8916,0.8794,-0.597527,0.5273,11.590,0.9740,12.664,0.003314,0.002817,0.008790,0.005371,0


In [5]:
train.shape

(458913, 13)

In [6]:
test.head()

,customer_ID,P_2_mean,P_2_median,P_2_sum,P_2_correlation,P_3_mean,S_25_mean,S_25_sum,S_25_std,S_25_mad,S_25_data_range,S_25_iqr
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.6016,0.5970,5.414,-0.484413,0.5737,0.9740,8.766,0.003436,0.003038,0.00879,0.005859
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.8623,0.8610,11.210,-0.459726,0.5530,0.9740,12.664,0.003073,0.002554,0.00830,0.005371
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.7490,0.7437,9.734,-0.398301,0.6710,0.9730,12.650,0.003055,0.002628,0.00830,0.005371
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.4746,0.4740,6.170,0.511864,0.6110,0.9730,12.650,0.003233,0.002855,0.00879,0.005859
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.3240,0.3162,4.215,-0.495118,0.6350,0.5396,7.016,0.363472,0.334700,0.81640,0.712158


In [7]:
test.shape

(589833, 12)

## Testing:

In [10]:
## Importing libraries
import boto3
import pandas as pd
import numpy as np
import miceforest as mf

## -------------------------------------------

## Sanity check
print('-- Process Starting --')

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'evan-callaghan-bucket'
bucket = s3.Bucket(bucket_name)

file_key = 'Kaggle-American-Express-Default/amex_test_data.csv'
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Creating data-type dictionary for reading the train data-frame
dtype_dict = {'customer_ID': "object", 'S_2': "object", 'P_2': 'float16', 'D_39': 'float16', 'B_1': 'float16','B_2': 'float16', 'R_1': 'float16','S_3': 'float16','D_41': 'float16','B_3': 'float16','D_42': 'float16','D_43': 'float16','D_44': 'float16', 'B_4': 'float16','D_45': 'float16','B_5': 'float16','R_2': 'float16','D_46': 'float16','D_47': 'float16','D_48': 'float16', 'D_49': 'float16','B_6': 'float16','B_7': 'float16','B_8': 'float16','D_50': 'float16','D_51': 'float16','B_9': 'float16', 'R_3': 'float16','D_52': 'float16','P_3': 'float16','B_10': 'float16','D_53': 'float16','S_5': 'float16','B_11': 'float16', 'S_6': 'float16','D_54': 'float16','R_4': 'float16','S_7': 'float16','B_12': 'float16','S_8': 'float16','D_55': 'float16', 'D_56': 'float16','B_13': 'float16','R_5': 'float16','D_58': 'float16','S_9': 'float16','B_14': 'float16','D_59': 'float16', 'D_60': 'float16','D_61': 'float16','B_15': 'float16','S_11': 'float16','D_62': 'float16','D_63': 'object','D_64': 'object', 'D_65': 'float16','B_16': 'float16','B_17': 'float16','B_18': 'float16','B_19': 'float16','D_66': 'float16','B_20': 'float16', 'D_68': 'float16','S_12': 'float16','R_6': 'float16','S_13': 'float16','B_21': 'float16','D_69': 'float16','B_22': 'float16', 'D_70': 'float16','D_71': 'float16','D_72': 'float16','S_15': 'float16','B_23': 'float16','D_73': 'float16','P_4': 'float16', 'D_74': 'float16','D_75': 'float16','D_76': 'float16','B_24': 'float16','R_7': 'float16','D_77': 'float16','B_25': 'float16', 'B_26': 'float16','D_78': 'float16','D_79': 'float16','R_8': 'float16','R_9': 'float16','S_16': 'float16','D_80': 'float16', 'R_10': 'float16','R_11': 'float16','B_27': 'float16','D_81': 'float16','D_82': 'float16','S_17': 'float16','R_12': 'float16', 'B_28': 'float16','R_13': 'float16','D_83': 'float16','R_14': 'float16','R_15': 'float16','D_84': 'float16','R_16': 'float16', 'B_29': 'float16','B_30': 'float16','S_18': 'float16','D_86': 'float16','D_87': 'float16','R_17': 'float16','R_18': 'float16', 'D_88': 'float16','B_31': 'int64','S_19': 'float16','R_19': 'float16','B_32': 'float16','S_20': 'float16','R_20': 'float16', 'R_21': 'float16','B_33': 'float16','D_89': 'float16','R_22': 'float16','R_23': 'float16','D_91': 'float16','D_92': 'float16', 'D_93': 'float16','D_94': 'float16','R_24': 'float16','R_25': 'float16','D_96': 'float16','S_22': 'float16','S_23': 'float16', 'S_24': 'float16','S_25': 'float16','S_26': 'float16','D_102': 'float16','D_103': 'float16','D_104': 'float16','D_105': 'float16', 'D_106': 'float16','D_107': 'float16','B_36': 'float16','B_37': 'float16', 'R_26': 'float16','R_27': 'float16','B_38': 'float16', 'D_108': 'float16','D_109': 'float16','D_110': 'float16','D_111': 'float16','B_39': 'float16','D_112': 'float16','B_40': 'float16', 'S_27': 'float16','D_113': 'float16','D_114': 'float16','D_115': 'float16','D_116': 'float16','D_117': 'float16','D_118': 'float16', 'D_119': 'float16','D_120': 'float16','D_121': 'float16','D_122': 'float16','D_123': 'float16','D_124': 'float16','D_125': 'float16', 'D_126': 'float16','D_127': 'float16','D_128': 'float16','D_129': 'float16','B_41': 'float16','B_42': 'float16','D_130': 'float16', 'D_131': 'float16','D_132': 'float16','D_133': 'float16','R_28': 'float16','D_134': 'float16','D_135': 'float16','D_136': 'float16', 'D_137': 'float16','D_138': 'float16','D_139': 'float16','D_140': 'float16','D_141': 'float16','D_142': 'float16','D_143': 'float16', 'D_144': 'float16','D_145': 'float16'}

## Reading the data
test = pd.read_csv(file_content_stream, dtype = dtype_dict)

-- Process Starting --


In [11]:
test.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631348,0.001912,0.010727,0.814453,0.007545,0.168701,0.009972,0.002348,...,NaN,NaN,NaN,NaN,0.004669,NaN,NaN,NaN,0.008278,NaN
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.586914,0.005276,0.011024,0.811035,0.001817,0.241333,0.000166,0.009132,...,NaN,NaN,NaN,0.000142,0.004940,0.009018,NaN,0.003695,0.003754,0.001460
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.608887,0.003326,0.016388,1.004883,0.000114,0.267090,0.004196,0.004192,...,NaN,NaN,NaN,0.000074,0.002113,0.004658,NaN,0.003155,0.002155,0.006481
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-05-20,0.614746,0.009064,0.021667,0.816406,0.009720,0.188965,0.004124,0.015327,...,NaN,NaN,NaN,0.004742,0.006393,0.002890,NaN,0.006042,0.005207,0.007858
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-06-15,0.591797,0.238770,0.015930,0.810547,0.002026,0.180054,0.000731,0.011284,...,NaN,NaN,NaN,0.008133,0.004330,0.008385,NaN,0.001008,0.007420,0.009468


In [13]:
test.shape

(7249445, 190)

In [ ]:
## Sanity check
print('-- Data Read --')

## Subsetting the data for Payment and Spend variables
test = test[['customer_ID', 'P_2', 'P_3', 'P_4', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 
               'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']]

print(test.columns)

In [12]:
test['customer_ID'].nunique()

589833

In [14]:
test.groupby('customer_ID')[['P_2', 'P_3']].mean().shape

(589833, 2)

In [ ]:
## Imputing the test data-frame using the Mice Forest library

## Defining the input variables and dropping customer_ID
mf_test = test.drop(columns = ['customer_ID'])

# Building the miceforest kernel
kernel_test = mf.ImputationKernel(mf_test, datasets = 5, save_all_iterations = True)

## Assigning the final imputed data-frames
test_impute = kernel_test.complete_data(dataset = 0, inplace = False)

## Adding "customer_ID" back into the data-frames
test_impute = pd.concat([test[['customer_ID']], test_impute], axis = 1)

## Sanity check
print('-- Testing data-frame imputation complete -- \n')

In [ ]:
## Creating other aggregated features of interest

## Sanity check
print('-- Testing data-frame aggregations starting -- \n')

## Creating a series of aggregation functions
def data_range(x):
    return x.max() - x.min()
def iqr(x):
    return np.percentile(x, 75) - np.percentile(x, 25)
def correlation(x):
    return pd.Series(x.values).corr(other = pd.Series(x.index), method = 'pearson')

## Creating new Payment features with the cleaned test data-frame
payment_vars_test = test_impute.groupby('customer_ID').agg({'P_2':['mean', 'median', 'sum', correlation], 'P_3':['mean']}).reset_index(drop = False)

## Renaming variable names
payment_vars_test.columns = ['customer_ID', 'P_2_mean', 'P_2_median', 'P_2_sum', 'P_2_correlation', 'P_3_mean']

## Creating new Spend features with the cleaned test data-frame
spend_vars_test = test_impute.groupby('customer_ID').agg({'S_25':['mean', 'sum', 'std', 'mad', data_range, iqr]}).reset_index(drop = False)

spend_vars_test.columns = ['customer_ID', 'S_25_mean', 'S_25_sum', 'S_25_std', 'S_25_mad', 'S_25_data_range', 'S_25_iqr']

## Sanity check
print('-- Testing aggregations complete -- \n')

In [ ]:
## Combining desired count features with other aggregated features to create final data-frame

## Joining the Payment and Spend train data-frames
testing = payment_vars_test.merge(spend_vars_test, how = 'left', on = 'customer_ID')

## -------------------------------------------

## Exporting the resulting training data-frame to a csv file
testing.to_csv('amex_test_payment_spend_final.csv', index = False)

## Sanity check
print('-- Testing data-frame complete -- \n')

## Modeling Processes:

#### Splitting the data

In [6]:
## Splitting the train data-frame into training (80%) and validation (20%)

## Defining the input and target variables
X_train = train.drop(columns = ['customer_ID', 'target'])
X_test = test.drop(columns = ['customer_ID'])
Y_train = train['target']

## Splitting the data
X_training, X_validation, Y_training, Y_validation = train_test_split(X_train, Y_train, test_size = 0.2, stratify = Y_train)

#### Hyper-parameter tuning with Optuna

In [7]:
## XGBoost Classifier
def objective_amex_xgb(trial):
    
    ## Defining the XGB hyper-parameter grid
    XGB_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 500, 100),
                     'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.951, step = 0.05),
                     'min_split_loss': trial.suggest_int('min_split_loss', 0, 5, 1),
                     'max_depth' : trial.suggest_int('max_depth', 3, 7, 1),
                     'min_child_weight' : trial.suggest_int('min_child_weight', 5, 9, 1),
                     'subsample' : trial.suggest_float('subsample', 0.6, 1, step = 0.1),
                     'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.6, 1, step = 0.1)}
    
    ## Building the XGBClassifier model
    model = XGBClassifier(**XGB_param_grid, n_jobs = -1).fit(X_training, Y_training)
        
    ## Predicting on the validation data-frame
    xgb_val_preds = model.predict_proba(X_validation)[:, 1]
    
    ## Evaluating model performance on the validation set
    amex_score = amex_metric(Y_validation, xgb_val_preds)
    
    ## Returning absolute difference of model validation predictions
    return amex_score


## RandomForest Classifier
def objective_amex_rf(trial):
    
    ## Defining the XGB hyper-parameter grid
    rf_param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 500, 100), 
                     'max_depth' : trial.suggest_int('max_depth', 3, 7, 1),
                     'min_samples_split' : trial.suggest_int('min_samples_split', 5, 15, 1), 
                     'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 5, 15, 1)}
    
    ## Building the RandomForestClassifier model
    model = RandomForestClassifier(**rf_param_grid, n_jobs = -1).fit(X_training, Y_training)
        
    ## Predicting on the validation data-frame
    rf_val_preds = model.predict_proba(X_validation)[:, 1]
    
    ## Evaluating model performance on the validation set
    amex_score = amex_metric(Y_validation, rf_val_preds)
    
    ## Returning absolute difference of model validation predictions
    return amex_score

In [8]:
## Calling Optuna objective function for XGBoost
xgb_study = optuna.create_study(direction = 'maximize')
xgb_study.optimize(objective_amex_xgb, n_trials = 5)

## Extracting best model 
xgb_best_params = xgb_study.best_trial.params

[I 2022-07-24 19:16:38,185] A new study created in memory with name: no-name-d1bc5d7b-0ddf-46dd-92ff-2fd97598f7cc
[I 2022-07-24 19:17:20,932] Trial 0 finished with value: 9.124367584192536e-05 and parameters: {'n_estimators': 300, 'learning_rate': 0.101, 'min_split_loss': 2, 'max_depth': 5, 'min_child_weight': 7, 'subsample': 0.8, 'colsample_bytree': 0.6}. Best is trial 0 with value: 9.124367584192536e-05.
[I 2022-07-24 19:17:33,246] Trial 1 finished with value: 0.00027461626555567997 and parameters: {'n_estimators': 100, 'learning_rate': 0.30100000000000005, 'min_split_loss': 5, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.6, 'colsample_bytree': 0.9}. Best is trial 1 with value: 0.00027461626555567997.
[I 2022-07-24 19:18:01,368] Trial 2 finished with value: 0.0004714649529968494 and parameters: {'n_estimators': 300, 'learning_rate': 0.7010000000000001, 'min_split_loss': 4, 'max_depth': 3, 'min_child_weight': 6, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 2 with

In [9]:
## Calling Optuna objective function for Random Forest
rf_study = optuna.create_study(direction = 'maximize')
rf_study.optimize(objective_amex_rf, n_trials = 5)

## Extracting best model 
rf_best_params = rf_study.best_trial.params

[I 2022-07-24 19:19:36,854] A new study created in memory with name: no-name-8efbd714-701c-4b80-b66b-7a5d1fe38088
[I 2022-07-24 19:19:50,920] Trial 0 finished with value: 0.00047763268668644596 and parameters: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.00047763268668644596.
[I 2022-07-24 19:20:47,148] Trial 1 finished with value: 0.0003671679426219543 and parameters: {'n_estimators': 400, 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.00047763268668644596.
[I 2022-07-24 19:21:18,361] Trial 2 finished with value: 0.00039576080194453703 and parameters: {'n_estimators': 200, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.00047763268668644596.
[I 2022-07-24 19:22:11,143] Trial 3 finished with value: 0.00019163317026102758 and parameters: {'n_estimators': 300, 'max_depth': 7, 'min_samples_split': 5, 'min_samples_leaf': 7}. Be

#### Modeling

In [ ]:
## Building a Random Forest model with the optimal set of hyper-parameters
rf_md = RandomForestClassifier(**rf_best_params, n_jobs = -1).fit(X_training, Y_training)

## Predicting on the test data-frame
X_test_preds = rf_md.predict_proba(X_test)[:, 1]

## Creating the final output data-frame
data_out = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': X_test_preds})

## Exporting as a csv file for submission
data_out.to_csv('amex_rf_payment_spend_preds.csv', index = False)

In [ ]:
## Building a XGBoost model with the optimal set of hyper-parameters
xgb_md = XGBClassifier(**xgb_best_params, n_jobs = -1).fit(X_training, Y_training)

## Predicting on the test data-frame
X_test_preds = xgb_md.predict_proba(X_test)[:, 1]

## Creating the final output data-frame
data_out = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': X_test_preds})

## Exporting as a csv file for submission
data_out.to_csv('amex_xgb_payment_spend_preds.csv', index = False)